In [16]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install sklearn_crfsuite

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-rvn9p3_v
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-rvn9p3_v
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101065 sha256=a3179107ccd8e3f3adb522f7448e3ffb84ace08b216664a8f5ab90add05daff4
  Stored in directory: /tmp/pip-ephem-wheel-cache-gvxqva61/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


## NER TASK 3 MODELS

In [19]:
import json
import pandas as pd
import numpy as np
import re
import string
from keras.models import Model, Input,Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import keras as k
from keras_contrib.layers import CRF

In [18]:
idx2tag = {0: 'I_DATE', 1: 'B_WITNESS', 2: 'I_ORG', 3: 'B_PETITIONER', 4: 'B_OTHER_PERSON', 5: 'I_WITNESS', 6: 'B_ORG', 7: 'I_PRECEDENT', 8: 'B_COURT', 9: 'B_RESPONDENT', 10: 'I_PROVISION', 11: 'I_CASE_NUMBER', 12: 'I_COURT', 13: 'I_GPE', 14: 'B_PRECEDENT', 15: 'B_JUDGE', 16: 'O', 17: 'I_OTHER_PERSON', 18: 'I_STATUTE', 19: 'B_DATE', 20: 'B_CASE_NUMBER', 21: 'I_PETITIONER', 22: 'I_JUDGE', 23: 'B_STATUTE', 24: 'B_PROVISION', 25: 'B_GPE', 26: 'I_RESPONDENT'}

In [9]:
# LOAD MODEL
import joblib
X_test,y_test = joblib.load('/kaggle/input/testing/Trained_Models/Test_data_part_3/test_data.pkl')


#### GLOVE

In [20]:
from tensorflow.keras.models import load_model
from keras.models import load_model

def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "crf_loss": loss, "crf_viterbi_accuracy":accuracy}

def load_keras_model(path):
    model = load_model(path, custom_objects=create_custom_objects())
    return model

model = load_keras_model('/kaggle/input/testing/Trained_Models/Task 1/GloVe/t1_model4_glove.h5')
print(model.summary)
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
print(len(X_test))
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

/opt/conda/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7d28aec9a0f0>>
949
949/949 [==============================] - 2s 2ms/step
                precision    recall  f1-score   support

 B_CASE_NUMBER       0.88      0.86      0.87       326
       B_COURT       0.06      0.22      0.09         9
        B_DATE       0.81      0.63      0.71       177
         B_GPE       0.72      0.22      0.34        58
       B_JUDGE       0.99      0.99      0.99     66228
         B_ORG       0.84      0.93      0.88       102
B_OTHER_PERSON       0.60      0.67      0.64       121
  B_PETITIONER       0.17      0.67      0.27         9
   B_PRECEDENT       0.71      0.62      0.67         8
   B_PROVISION       0.91      0.89      0.90       178
  B_RESPONDENT       0.71      0.93      0.80       344
     B_STATUTE       0.53      0.42      0.47       159
     B_WITNESS       0.89      0.90      0.90       383
 I_CASE_NUMBER       0.97      0.87      0.92       439
    

#### WORD 2 VEC

In [21]:
from tensorflow.keras.models import load_model
from keras.models import load_model

def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "crf_loss": loss, "crf_viterbi_accuracy":accuracy}

def load_keras_model(path):
    model = load_model(path, custom_objects=create_custom_objects())
    return model

model = load_keras_model('/kaggle/input/testing/Trained_Models/Task 1/Word2Vec/t1_model4_word2vec.h5')

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
print(len(X_test))
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

/opt/conda/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


949
949/949 [==============================] - 2s 2ms/step
                precision    recall  f1-score   support

 B_CASE_NUMBER       0.77      0.88      0.82       326
       B_COURT       0.00      0.00      0.00         9
        B_DATE       0.74      0.66      0.70       177
         B_GPE       0.64      0.31      0.42        58
       B_JUDGE       0.99      0.99      0.99     66228
         B_ORG       0.70      0.92      0.80       102
B_OTHER_PERSON       0.62      0.57      0.59       121
  B_PETITIONER       0.38      0.33      0.35         9
   B_PRECEDENT       0.11      0.12      0.12         8
   B_PROVISION       0.94      0.87      0.90       178
  B_RESPONDENT       0.62      0.78      0.69       344
     B_STATUTE       0.64      0.48      0.55       159
     B_WITNESS       0.83      0.85      0.84       383
 I_CASE_NUMBER       0.91      0.83      0.87       439
       I_COURT       0.91      0.84      0.87      1793
        I_DATE       0.69      0.52      0.5

#### Fast Text

In [22]:
from tensorflow.keras.models import load_model
from keras.models import load_model

def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "crf_loss": loss, "crf_viterbi_accuracy":accuracy}

def load_keras_model(path):
    model = load_model(path, custom_objects=create_custom_objects())
    return model

model = load_keras_model('/kaggle/input/testing/Trained_Models/Task 1/Fasttext/t1_model4_fasttext.h5')

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
print(len(X_test))
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

/opt/conda/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


949
949/949 [==============================] - 2s 2ms/step
                precision    recall  f1-score   support

 B_CASE_NUMBER       0.77      0.80      0.78       326
       B_COURT       0.00      0.00      0.00         9
        B_DATE       0.55      0.43      0.48       177
         B_GPE       0.19      0.21      0.20        58
       B_JUDGE       0.99      0.98      0.99     66228
         B_ORG       0.45      0.87      0.60       102
B_OTHER_PERSON       0.50      0.42      0.46       121
  B_PETITIONER       0.12      0.22      0.15         9
   B_PRECEDENT       0.11      0.25      0.15         8
   B_PROVISION       0.82      0.84      0.83       178
  B_RESPONDENT       0.53      0.64      0.58       344
     B_STATUTE       0.51      0.39      0.44       159
     B_WITNESS       0.73      0.86      0.79       383
 I_CASE_NUMBER       0.90      0.80      0.85       439
       I_COURT       0.91      0.57      0.70      1793
        I_DATE       0.46      0.37      0.4

## ATE TASK 3 MODELS

In [23]:
tag2idx = {0: 'I', 1: 'B', 2: 'O'}

In [24]:
## Load data 
import joblib
X_test,y_test = joblib.load('/kaggle/input/testing/Trained_Models/Test_data_part_3/test_data_ate.pkl') 

#### Glove

In [25]:
from tensorflow.keras.models import load_model
from keras.models import load_model

def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "crf_loss": loss, "crf_viterbi_accuracy":accuracy}

def load_keras_model(path):
    model = load_model(path, custom_objects=create_custom_objects())
    return model

model = load_keras_model('/kaggle/input/testing/Trained_Models/Task 2/GloVe/t2_model4_glove.h5')
print(model.summary)
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
print(len(X_test))
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

/opt/conda/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7d28ae6ab978>>
328
328/328 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

   B_WITNESS       0.99      0.99      0.99     24878
      I_DATE       0.70      0.77      0.73       463
       I_ORG       0.67      0.53      0.59       243

    accuracy                           0.98     25584
   macro avg       0.79      0.76      0.77     25584
weighted avg       0.98      0.98      0.98     25584



#### Word2V

In [27]:
from tensorflow.keras.models import load_model
from keras.models import load_model

def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "crf_loss": loss, "crf_viterbi_accuracy":accuracy}

def load_keras_model(path):
    model = load_model(path, custom_objects=create_custom_objects())
    return model

model = load_keras_model('/kaggle/input/testing/Trained_Models/Task 2/Word2vec/t2_model4_word2vec.h5')
print(model.summary)
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
print(len(X_test))
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

/opt/conda/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7d28ace89a20>>
328
328/328 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

   B_WITNESS       0.99      0.99      0.99     24878
      I_DATE       0.64      0.74      0.68       463
       I_ORG       0.51      0.55      0.53       243

    accuracy                           0.98     25584
   macro avg       0.71      0.76      0.74     25584
weighted avg       0.98      0.98      0.98     25584



#### Fast Text

In [28]:
from tensorflow.keras.models import load_model
from keras.models import load_model

def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "crf_loss": loss, "crf_viterbi_accuracy":accuracy}

def load_keras_model(path):
    model = load_model(path, custom_objects=create_custom_objects())
    return model

model = load_keras_model('/kaggle/input/testing/Trained_Models/Task 2/Fasttext/t2_model4_fasttext.h5')
print(model.summary)
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
print(len(X_test))
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

/opt/conda/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7d28ac6395c0>>
328
328/328 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

   B_WITNESS       0.99      0.99      0.99     24878
      I_DATE       0.73      0.64      0.68       463
       I_ORG       0.55      0.52      0.54       243

    accuracy                           0.98     25584
   macro avg       0.76      0.72      0.74     25584
weighted avg       0.98      0.98      0.98     25584

